In [36]:
import graphviz

def generate_large_scale_wbs(data, filename='wbs'):
    """
    Generates a high-visibility WBS with large fonts and orthogonal tree lines.
    """
    dot = graphviz.Digraph(format='png')
    
    # Use ortho splines for 90-degree tree lines, reduced ranksep for tighter vertical spacing
    dot.attr(rankdir='TB', splines='ortho', nodesep='1.0', ranksep='0.6', overlap='false')
    # Use Century Gothic or similar sans-serif fallbacks
    dot.attr('node', fontname='Century Gothic, Futura, Arial, sans-serif')

    node_map = {} 
    nodes_by_depth = {}  # Track nodes at each depth for alignment
    leaf_groups_by_depth = {}  # Track leaf group sizes by depth
    task_id_map = {}  # Map task names to their full IDs

    def is_leaf_group(d):
        return all(not isinstance(v, dict) for v in d.values())

    def get_font_size(depth):
        sizes = {0: '32', 1: '24', 2: '20'}
        return sizes.get(depth, '20')

    def get_fill_color(depth):
        colors = {
            0: '#8497b0',
            1: '#adb9ca',
            2: '#d6dce5',
            3: '#e5e9ef',
        }
        return colors.get(depth, '#e5e9ef')

    # First pass: collect leaf group sizes and build task ID map
    def collect_info(content, prefix="", depth=0):
        items = list(content.items())
        for index, (key, value) in enumerate(items, 1):
            if depth == 0:
                current_id_str = ""
                next_prefix = ""
            else:
                current_id_str = f"{prefix}{index}"
                next_prefix = f"{current_id_str}."

            if isinstance(value, dict) and not is_leaf_group(value):
                collect_info(value, next_prefix, depth + 1)
            elif isinstance(value, dict) and is_leaf_group(value):
                num_items = len(value)
                if depth not in leaf_groups_by_depth:
                    leaf_groups_by_depth[depth] = []
                leaf_groups_by_depth[depth].append(num_items)
                
                # Map each task name to its full ID
                for sub_idx, task_name in enumerate(value.keys(), 1):
                    full_id = f"{current_id_str}.{sub_idx}" if current_id_str else str(sub_idx)
                    task_id_map[task_name] = full_id

    collect_info(data)
    
    # Calculate max items per depth level
    max_items_by_depth = {d: max(sizes) for d, sizes in leaf_groups_by_depth.items()}

    def get_dependency_str(dep_list):
        """Get formatted dependency string from list of task names."""
        if not dep_list:
            return ""
        dep_ids = []
        for dep_name in dep_list:
            if dep_name in task_id_map:
                dep_ids.append(task_id_map[dep_name])
        if dep_ids:
            return f"<BR/>Depends on [{', '.join(dep_ids)}]"
        return ""

    def add_nodes(parent_id, content, prefix="", depth=0):
        items = list(content.items())
        
        for index, (key, value) in enumerate(items, 1):
            if depth == 0:
                current_id_str = ""
                display_label = key
                node_id = "root"
                next_prefix = ""
            else:
                current_id_str = f"{prefix}{index}"
                display_label = f"{current_id_str}. {key}"
                node_id = f"n{current_id_str.replace('.', '_')}"
                next_prefix = f"{current_id_str}."

            if depth not in nodes_by_depth:
                nodes_by_depth[depth] = []
            nodes_by_depth[depth].append(node_id)

            fsize = get_font_size(depth)
            fill_color = get_fill_color(depth)
            
            if isinstance(value, dict) and not is_leaf_group(value):
                label = f'<<FONT POINT-SIZE="{fsize}"><B>{display_label}</B></FONT>>'
                dot.node(node_id, label, shape='rectangle', style='filled', fillcolor=fill_color, margin='0.4')
                if parent_id:
                    dot.edge(parent_id, node_id, penwidth='2.0', dir='none')
                add_nodes(node_id, value, next_prefix, depth + 1)
            
            elif isinstance(value, dict) and is_leaf_group(value):
                header_fsize = get_font_size(depth)
                header_color = get_fill_color(depth)
                row_color = get_fill_color(depth + 1)
                
                # Estimate width based on header text (roughly 10 pixels per character at font size 20)
                header_width = len(display_label) * 12
                min_width = max(200, header_width)  # Minimum 200 pixels
                
                table_label = f'<<TABLE BORDER="0" CELLBORDER="1" CELLSPACING="0" CELLPADDING="25" FIXEDSIZE="TRUE" WIDTH="{min_width}">'
                table_label += f'<TR><TD BGCOLOR="{header_color}" WIDTH="{min_width}"><B><FONT POINT-SIZE="{header_fsize}">{display_label}</FONT></B></TD></TR>'
                
                num_items = len(value)
                max_items = max_items_by_depth.get(depth, num_items)
                
                for sub_idx, (task_name, task_info) in enumerate(value.items(), 1):
                    port_id = f"p{sub_idx}"
                    full_id = f"{current_id_str}.{sub_idx}" if current_id_str else str(sub_idx)
                    
                    # Parse task_info: can be "name", ("name",), or ("name", [deps])
                    if isinstance(task_info, tuple):
                        who = task_info[0]
                        deps = task_info[1] if len(task_info) > 1 else []
                    else:
                        who = task_info
                        deps = []
                    
                    dep_str = get_dependency_str(deps)
                    
                    table_label += f'<TR><TD PORT="{port_id}" ALIGN="LEFT" BALIGN="LEFT" BGCOLOR="{row_color}" WIDTH="{min_width}">'
                    table_label += f'<FONT POINT-SIZE="18"><B>{full_id}. {task_name}</B><BR/><I>[{who}]{dep_str}</I></FONT>'
                    table_label += '</TD></TR>'
                    node_map[task_name] = (node_id, port_id)
                
                # Add invisible padding rows to match the tallest table at this depth
                padding_needed = max_items - num_items
                for _ in range(padding_needed):
                    table_label += f'<TR><TD BORDER="0" WIDTH="{min_width}">'
                    table_label += f'<FONT POINT-SIZE="18" COLOR="white"><B>padding</B><BR/><I>padding</I></FONT>'
                    table_label += '</TD></TR>'
                
                table_label += '</TABLE>>'
                dot.node(node_id, table_label, shape='none')
                if parent_id:
                    dot.edge(parent_id, node_id, penwidth='2.0', dir='none')

    # Build hierarchy
    add_nodes(None, data)

    # Force same-depth nodes into same rank
    for depth, node_ids in nodes_by_depth.items():
        if len(node_ids) > 1:
            with dot.subgraph() as s:
                s.attr(rank='same')
                for nid in node_ids:
                    s.node(nid)

    dot.render(filename, cleanup=True)
    print(f"High-visibility WBS generated: {filename}.png")

# Data definitions
# Task format: "Task Name": "Who"  OR  "Task Name": ("Who", ["Dependency1", "Dependency2"])
wbs_hierarchy = {
    "S26-14 Quadrature Inclinometer": {
        "Assembly": {
            "Component Sourcing": {
                "Update PCB layout": "Timberly",
                "Source SMD resistors, capacitors, and inductors": "Alex",
            },
            "Hardware": {
                "Test MCU": "Richard",
                "Test Accelerometer": "All",
                "Test Voltage Regulator": "All",
                "Test Gyroscope": "All",
                "Test LEDs/buttons": "All",
                "Verify PCB integrity": "Max",
                "Solder components": ("Max", ["Test MCU", "Test Accelerometer", "Test Voltage Regulator", "Test Gyroscope", "Test LEDs/buttons", "Verify PCB integrity"]),
            },
            "Software": {
                "Develop MCU control": "Richard, Molly",
                "Implement Kalman filter": "Richard, Max",
                "Develop calibration software": "Richard, Max",
            }
        },
        "Validation and Testing": {
            "Internal Development": {
                "Develop arduino decoder": "Alex",
                "Create tilting system": "All",
            },
            "System Verification": {
                "Verify power delivery": ("Timberly", ["Solder components"]),
                "Verify sensor readings": ("Alex/Max", ["Solder components", "Develop MCU control"]),
                "Verify quadrature output": ("Alex", ["Verify sensor readings", "Develop arduino decoder"]),
                "Verify calibration procedure": ("Alex", ["Develop MCU control", "Develop calibration software"]),
                "Verify angular accuracy": ("Max", ["Verify calibration procedure"]),
            }
        },
        "Presentation": {
            "Project Deliverables": {
                "Create final presentation": ("All", ["Verify angular accuracy"]),
                "Create poster": ("All", ["Verify angular accuracy"])
            }
        }
    }
}

if __name__ == "__main__":
    generate_large_scale_wbs(wbs_hierarchy)

in label of node n1_1
in label of node n1_2
in label of node n1_3
in label of node n2_1
in label of node n2_2
in label of node n3_1


High-visibility WBS generated: wbs.png
